# Fundamentos de procesamiento digital de imágenes

### Prof. Arturo González Vega
### Alumna: Salma Patricia Gutiérrez Rivera

## Segundo examen parcial

__1. Haga una función llamada ImPotencia que, dada una imagen, calcule su potencia promedio. Para realizar el cálculo utilice el teorema de Parseval que dice lo siguiente:__

$$
P_f = \frac{1}{MN}\sum_u\sum_v|F(u,v)|^2
$$

In [8]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from math import exp

In [2]:
def magnitud_dft(imagen):
    """Esta función solo sirve con imágenes BGR,
    aunque sean a escala de grises. Usa cv2.imread(imagen, 0)"""
    dft = cv2.dft(np.float32(imagen), flags = cv2.DFT_COMPLEX_OUTPUT) # Transformada de la imagen
    dft_shift = np.fft.fftshift(dft) # Centramos la transformada
    magnitud = cv2.magnitude(dft_shift[:,:,0], dft_shift[:,:,1]) # Magnitud del espectro
    
    return magnitud

In [3]:
def ImPotencia(espectro_dft):
    M, N = imagen.shape
    potencia = np.sum(espectro * espectro) / (M * N)
    
    return potencia

__2. Haga una función FiltraGaussiana que, dada una imagen, el tipo de filtro (pasabajos o pasaaltos) y el valor de $\sigma$, realice un filtrado con una función Gaussiana con la desviación indicada.__

In [10]:
def FiltraGaussiana(imagen_dft, sigma, omega = 0):
    P, Q = imagen_dft.shape
    img_filtrada = np.zeros(P, Q)
    
    if omega == 0: # Filtro pasabajos
        for u in range(P):
            for v in range(Q):
                D_cuadrada = (u - 0.5 * P)**2 + (v - 0.5 * Q)**2
                img_filtrada = exp(- 0.5 * D_cuadrada / sigma**2)
    else: # Filtro pasaaltos
        for u in range(P):
            for v in range(Q):
                D_cuadrada = (u - 0.5 * P)**2 + (v - 0.5 * Q)**2
                img_filtrada = 1.0 - exp(- (D_cuadrada - sigma**2)**2 / (D_cuadrada * omega**2))
                
    return img_filtrada

In [ ]:
img = cv2.imread('FigP0401(test_pattern).tif', 0)

dft = cv2.dft(np.float32(img), flags = cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(dft)

magnitude_spectrum = 20 * np.log(cv2.magnitude(dft_shift[:,:,0], dft_shift[:,:,1]))

plt.subplot(121), plt.imshow(img, cmap = 'gray')
plt.title('Input Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(magnitude_spectrum, cmap = 'gray')
plt.title('Magnitude Spectrum'), plt.xticks([]), plt.yticks([])
plt.show()